## Generador de acrónimos 

Este ejemplo de jupyter notebook permite generar acrónimos a partir de una cadena de texto proporcionada como entrada y un corpus de palabras.

El algoritmo devuelve palabras que están contenidas en el título, de forma que al menos una letra pertenece a cada palabra, siempre y
cuando la palabra no sea un stopword (por ejemplo, "of"), en cuyo caso es opcional incluir o no una de sus letras en el acrónimo.

Por defecto, no distingue minúsculas y mayúsculas. La distinción entre mayúsculas y minúsculas implica que solo las letras mayúsculas son consideradas para el alineamiento.

Así, por ejemplo si tenemos en cuenta las mayúsculas y proporcionamos la cadena de texto "Platform for OPEN DATA ACCESS IN DIGITAL HUMANITIES RESEARCH", el proceso fuerza 
fuerza a que el acrónimo empiece por P y a que ningún término "for" sea incluido. En cambio, pueden aparecer o no letras de "IN" por ser un stopword.

Si no tenemos en cuenta la distinción entre mayúsculas y minúsculas, en realidad todo el título se transforma a mayúsculas.

In [27]:
import sys     

## Clase Vocabulary para almacenar el diccionario proporcionado como entrada.

Leemos línea a línea el fichero que recibimos como parámetro. Cada línea incluye una palabra a la que le quitamos los espacios de la izquierda y derecha y convertimos a mayúscula:

In [2]:
# Un conjunto de nombres (palabras en mayúsculas) que son acrónimos válidos
class Vocabulary(set):
    def __init__(self, filename):
        with open(filename, 'r') as f:
            for w in f: # lee linea a linea
                self.add(w.strip().upper()) # eliminamos espacios y convertimos a mayusculas

## ¿Cómo se utiliza la clase Vocabulary?
Para utilizar la clase Vocabulary es necesario el siguiente código que lee el fichero en_words.txt de la carpeta input:

In [3]:
words = Vocabulary('input/en_words.txt')

## La clase Title 

La clase Title almacena una cadena de texto que incluye varios términos. Contemplamos el caso de distinción entre mayúsculas y minúsculas.

En el caso de tener en cuenta las mayúsculas, únicamente los términos en mayúscula se tendrán en cuenta. Por ejemplo, en el caso de utilizar la cadena "First PLAN", solo se tendrá en cuenta la F del primer término y P, L, A, o N del segundo.
 

In [4]:
class Title(str):
    def __new__(cls, content, case_sensitive=True):
        
        # reducimos posibles espacios consecutivos a uno solo
        reduced = ' '.join(content.split())
        
        # contemplamos la distinción de mayúsculas y minúsculas 
        if case_sensitive:
            return super().__new__(cls, reduced)
            str.__init__(reduced)
        else:
            return super().__new__(cls, reduced.upper())
    
    # En el caso de tener en cuenta las mayúsculas, únicamente los términos en mayúscula se tendrán en cuenta. 
    # Por ejemplo, en el caso de utilizar la cadena "First PLAN", solo se tendrá en cuenta la F del primer término y P, L, A, o N del segundo.
    def __init__(self, content, case_sensitive=True):
        self._tokens = self.split()

        # marcamos la posición de los términos dependiendo del número de espacios precedentes
        self._token_number = {n:self[:n].count(' ') for n in range(len(self)) if self[n] != ' '}

    
    # devuelve la lista de términos en el titulo
    def tokens(self):
        return self._tokens
    
    # devuelve la posición del término en el titulo
    #return the token number for the specified position in the title
    def token_number(self, pos):
        return self._token_number[pos]
    
    # Devuelve verdadero si el término es una subsecuencia del titulo,
    # el término es el resultado de eliminar algunos caracteres (o ninguno) en el titulo
    def contains(self, word):
        n = -1
        for c in word:
            n = self.find(c, n + 1) 
            if n < 0:
                return False
        
        return True
        
    # Devuelve todos los posibles alineamientos de la palabra y el titulo.
    # Un alineamiento es una tupla a = (a1, a2, .., aN) donde a1 < a2 < .. < aN y word[k] = title[ak] 
    # Por ejemplo, si el término es 'AB' y el texto es 'ABAB', los alineamientos seran (0, 1), (0, 3) y (2,3)
    def all_alignments(self, word):
        A = [list(), list()]
        wsize = len(word)
        tsize = len(self)
        for j in range(1 + tsize):
            A[0].append(set())
        
        for i in range(1, 1 + wsize):
            A[i % 2] = [set()]
            for j in range(1, 1 + tsize):
                a = A[i%2][j - 1].copy()
                A[i % 2].append(a)
                if word[i - 1] == self[j - 1]: # add j - 1 to the tuples
                    A[i % 2][j].add((j - 1,)) 
                    for a in A[(i - 1) % 2][j - 1]:
                        A[i % 2][j].add(a + (j - 1,))
            
        # return only full aligments (word is exhausted and all chars matched)
        return set(a for a in A[wsize % 2][-1] if len(a) == len(word))


## Vamos a probar el generador de acrónimos

In [6]:
case_sensitive = True

title = Title('Platform for OPEN DATA ACCESS in DIGITAL HUMANITIES RESEARCH', case_sensitive)
print(title)
    
words = Vocabulary('input/en_words.txt')
print("lexicon has", len(words), 'words')

# los stopwords se pueden tener en cuenta opcionalmente
stopwords = Vocabulary('input/en_stopwords.txt')
lowercase = {token for token in title.tokens() if token.islower()}

# terminos en el titulo que no es necesario tener en cuenta
ignore = {n for n, token in enumerate(title.tokens()) if token.upper() in stopwords|lowercase}

for word in sorted(words):
    if title.contains(word):
        alignments = title.all_alignments(word)
        for a in alignments:
            matched = {title.token_number(pos) for pos in a}
            if len(ignore | matched) == len(title.tokens()):
                res = [c.upper() if n in a else c.lower() for n, c in enumerate(title)]
                print(word, ': ', ''.join(res))
                break

Platform for OPEN DATA ACCESS in DIGITAL HUMANITIES RESEARCH
lexicon has 370103 words
PEACELESS :  Platform for opEn datA aCcEss in digitaL humanitiES reSearch
PEASANTESS :  Platform for opEn datA acceSs in digitAl humaNiTiES reSearch
PEASANTS :  Platform for opEn datA acceSs in digitAl humaNiTies reSearch
PEDAGESE :  Platform for opEn Data Access in diGital humanitiES resEarch
PEDALER :  Platform for opEn Data Access in digitaL humanitiEs Research
PEDALIER :  Platform for opEn Data Access in digitaL humanItiEs reseaRch
PEDALIERS :  Platform for opEn Data Access in digitaL humanItiEs ReSearch
PEDALITER :  Platform for opEn Data Access in digitaL humanITiEs reseaRch
PEDETES :  Platform for opEn Data accEss in digiTal humanitiEs reSearch
PEDETIC :  Platform for opEn Data accEss in digiTal humanitIes researCh
PENTACETATE :  Platform for opEN daTa AcCEss in digiTal humAniTies rEsearch
PENTACLES :  Platform for opEN daTA acCess in digitaL humanitiEs reSearch
PENTADIC :  Platform for opEN da

## Vamos a probar sin distinguir entre mayúsculas y minúsculas y comprobamos el resultado

In [7]:
case_sensitive = False

title = Title('Platform for OPEN DATA ACCESS in DIGITAL HUMANITIES RESEARCH', case_sensitive)
print(title)
    
words = Vocabulary('input/en_words.txt')
print("lexicon has", len(words), 'words')

# los stopwords se pueden tener en cuenta opcionalmente
stopwords = Vocabulary('input/en_stopwords.txt')
lowercase = {token for token in title.tokens() if token.islower()}

# terminos en el titulo que no es necesario tener en cuenta
ignore = {n for n, token in enumerate(title.tokens()) if token.upper() in stopwords|lowercase}

for word in sorted(words):
    if title.contains(word):
        alignments = title.all_alignments(word)
        for a in alignments:
            matched = {title.token_number(pos) for pos in a}
            if len(ignore | matched) == len(title.tokens()):
                res = [c.upper() if n in a else c.lower() for n, c in enumerate(title)]
                print(word, ': ', ''.join(res))
                break

PLATFORM FOR OPEN DATA ACCESS IN DIGITAL HUMANITIES RESEARCH
lexicon has 370103 words
AEACIDES :  plAtform for opEn dAta aCcess In Digital humanitiEs reSearch
AFFRONTEDNESS :  plAtForm FoR OpeN daTa accEss in Digital humaNitiES reSearch
ANACAHUITA :  plAtform for opeN datA acCess in digitAl HUmanITies reseArch
ANACAHUITE :  plAtform for opeN dAta acCess in digitAl HUmanITies rEsearch
ANACANTH :  plAtform for opeN dAta aCcess in digitAl humaNiTies researcH
ANACLITIC :  plAtform for opeN datA acCess in digitaL humanITIes researCh
ANAESTHATIC :  plAtform for opeN dAta accESs in digiTal HumAniTIes researCh
ANASITCH :  plAtform for opeN dAta accesS in dIgital humaniTies researCH
ANASTALTIC :  plAtform for opeN dAta acceSs in digiTAL humaniTIes researCh
ANASTASES :  plAtform for opeN dAta accesS in digiTal humAnitieS rESearch
ANASTATE :  plAtform for opeN datA accesS in digiTAl humaniTies rEsearch
ANASTATIC :  plAtform for opeN dAta accesS in digiTAl humaniTIes researCh
ANDANTES :  plAtform 